## Yolo11 tests

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os , sys
import shutil
from tqdm.auto import tqdm  
sys.path.append('..')
from pathlib import Path
cur_path = Path(os.getcwd()).parent
sam2_path = cur_path / 'sam2_octron'
sys.path.append(cur_path.as_posix())
from matplotlib import pyplot as plt
import cmasher as cmr
import numpy as np
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'

In [ ]:
from napari_pyav._reader import FastVideoReader
from octron.sam2_octron.helpers.video_loader import get_vfile_hash
from octron.yolo_octron.helpers.training import (collect_labels, 
                                                 collect_polygons,
                                                 draw_polygons,
                                                 train_test_val,
                                                 write_training_data,
                                                 write_yolo_config_yaml,
                                                 
)
from octron.yolo_octron.helpers.polygons import (find_objects_in_mask, 
                                                 watershed_mask,
                                                 get_polygons,
)
from octron.sam2_octron.helpers.sam2_zarr import load_image_zarr

In [ ]:
# Presaved model (YOLOv11)
path_to_model = Path('/Users/horst/Documents/python/OCTRON/octron/yolo_octron/yolo11l-seg.pt')
project_path = Path('/Users/horst/Downloads/octron_project_2')
assert project_path.exists()
assert path_to_model.exists()

### Find out what info is present across object organizers / ...

- Construct a nice loop that finds the zarr, video files, and compares video file hashes 


In [ ]:
project_path

In [ ]:
label_dict = collect_labels(project_path, prune_empty_labels=True, min_num_frames=10, verbose=False)
label_dict = collect_polygons(label_dict)  

In [ ]:
# for folder, l in label_dict.items():
#     print(folder)
#     draw_polygons(l, l['video'], 
#                   max_to_plot=1,
#                   randomize=True
#                  )   
    

In [ ]:
# Perform the split of the data
# And save frames back into 'frames_split' key
print('Splitting the data for training into training, validation and testing fractions')  
for labels in label_dict.values():
    for entry in labels:
        if entry == 'video':
            continue    
        label = labels[entry]['label']
        print(f'Processing {label} ...')
        frames = labels[entry]['frames']   
        split_dict = train_test_val(frames, 
                                     training_fraction=0.7,
                                     validation_fraction=0.15,
                                     verbose=True
                                     )

        labels[entry]['frames_split'] = split_dict

In [ ]:
path_to_training = project_path / 'model'
try:
    path_to_training.mkdir(exist_ok=False)
except FileExistsError as e:
    # Check if path_to_training_data is empty
    if len(list(path_to_training.glob('*'))) > 1:
       raise FileExistsError(f'{path_to_training} is not empty. Please remove subfolders first.')
    else:
        pass

path_to_training_data = project_path / 'model' / 'training_data'
write_training_data(label_dict, path_to_training_data, verbose=True)

dataset_path = path_to_training_data
train_path = "train"  
val_path = "val"
test_path = "test"

# Get label names from the object organizer
label_id_label_dict = {}
for label_id, label_dict in labels.items():
    label_id_label_dict[label_id] = label_dict['label']

# Write the YAML config
config_path = path_to_training / "yolo_config.yaml"
_ = write_yolo_config_yaml(
    output_path=config_path,
    dataset_path=dataset_path,
    train_path=train_path,
    val_path=val_path,
    test_path=test_path,
    label_dict=label_id_label_dict
)

## YOLO setup

In [ ]:
from ultralytics import settings
settings.update({'sync':False,'hub':False,'runs_dir':path_to_training.as_posix()})
from ultralytics import YOLO

In [ ]:
# Load a model
import time
def on_train_start(trainer):
    print("🥳  Training is starting!")

def on_fit_epoch_end(trainer):
    current_epoch = trainer.epoch + 1 
    time_epoch = trainer.epoch_time
    print(f"Time for epoch: {time_epoch}")
    remaining_time = time_epoch * (no_epochs - current_epoch)   
    finish_time = time.time() + remaining_time
    print(f"Estimated time remaining: {remaining_time} seconds")    
    print(f'Estimated finish time: {time.ctime(finish_time)}')  
    
model = YOLO(path_to_model)  # load a pretrained model (recommended for training)
model.add_callback("on_train_start", on_train_start)
model.add_callback("on_fit_epoch_end", on_fit_epoch_end)
no_epochs = 30

# Train the model
# https://docs.ultralytics.com/usage/cfg/#solutions-settings
results = model.train(data=config_path, 
                      save_dir=path_to_training.as_posix(),
                      name='training',
                      mode='segment',
                      device='cpu',
                      mask_ratio=4,
                      epochs=no_epochs,
                      imgsz=640,
                      resume=False,
                      plots=True,
                      batch=.9,
                      cache=False,
                      save=True,
                      save_period=15,
                      project=None,
                      exist_ok=True,
                      # augmentation
                      augment=True,
                      hsv_v=.25,
                      degrees=180,
                      scale=.5,
                      shear=2,
                      flipud=.1,
                      fliplr=.1,
                      mosaic=1.0,
                      copy_paste=.5,
                      copy_paste_mode='mixup', 
                      erasing=.25,
                      crop_fraction=1.0,
                      )

In [ ]:
# model = YOLO('/Users/horst/Downloads/octron_project/octron_training/yolo runs/segment/train/weights/last.pt')  #

In [ ]:
# metrics = model.val(device='cpu', plots=True)

In [ ]:
# print("Mean Average Precision for boxes:", metrics.box.map)
# print("Mean Average Precision for masks:", metrics.seg.map)

In [ ]:
# # Run inference on 'bus.jpg' with arguments
# model.predict('/Users/horst/Downloads/octron_project/test data/8_behaviour_filtered2024-11-04T14_20_34_20240930_Th19.mp4', 
#               save=True, 
#               classes=[0],
#               imgsz=1000, 
#               device='cpu',
#               visualize=False,
#               conf=0.9
#               )

In [ ]:
# # Train/val/test sets as 
# 1) dir: path/to/imgs, 
# 2) file: path/to/imgs.txt, or list: [path/to/imgs1, path/to/imgs2, ..]
# path: ../datasets/coco8-seg # dataset root dir (absolute or relative; if relative, it's relative to default datasets_dir)
# train: images/train # train images (relative to 'path') 4 images
# val: images/val # val images (relative to 'path') 4 images
# test: # test images (optional)
